<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/Disaster_Tweets_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description 
This is a notebook on how to use pretrained BERT models to predict real or fake disaster tweets. 


In [24]:
!rm *
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
!wget -O tweets.zip ali.140714.xyz:8000/nlp-getting-started.zip 
!unzip tweets.zip 
!ls
!wget -O labels.csv ali.140714.xyz:8000/leaked_tweet_labels.csv

rm: cannot remove '__pycache__': Is a directory
rm: cannot remove 'sample_data': Is a directory
--2020-05-21 02:17:34--  http://ali.140714.xyz:8000/nlp-getting-started.zip
Resolving ali.140714.xyz (ali.140714.xyz)... 47.240.16.188
Connecting to ali.140714.xyz (ali.140714.xyz)|47.240.16.188|:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘tweets.zip’

tweets.zip          100%[===================>] 593.11K  --.-KB/s    in 0.07s   

2020-05-21 02:17:35 (8.45 MB/s) - ‘tweets.zip’ saved [607343/607343]

Archive:  tweets.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
__pycache__  sample_submission.csv  tokenization.py  tweets.zip
sample_data  test.csv		    train.csv
--2020-05-21 02:17:55--  http://ali.140714.xyz:8000/leaked_tweet_labels.csv
Resolving ali.140714.xyz (ali.140714.xyz)... 47.240.16.188
Connecting to ali.140714.xyz (ali.140714.xyz)|47.240.16

In [0]:
# !pip install sentencepiece
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

# Load data

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [27]:
%%time
import tensorflow_hub as hub 
import tokenization
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)
# bert_m = hub.Module(
#     "https://tfhub.dev/google/bert_cased_L-24_H-1024_A-16/1",
#     trainable=True, tags={"train"})

CPU times: user 4.08 s, sys: 551 ms, total: 4.63 s
Wall time: 4.56 s


In [0]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = Dense(1, activation='sigmoid')(net)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
max_len = 120
train_input = bert_encode(train.text.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.text.values, tokenizer, max_len=max_len)
train_labels = train.target.values

In [30]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 120)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 120)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 120)]        0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [31]:
%%time
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    train_input, train_labels, 
    validation_split=0.2,
    epochs=30,
    callbacks=[checkpoint, earlystopping],
    batch_size=16,
    verbose=1
)

Epoch 1/30
381/381 [==============================] - ETA: 0s - loss: 0.4838 - accuracy: 0.7757
Epoch 00001: val_accuracy improved from -inf to 0.83979, saving model to model.h5
381/381 [==============================] - 102s 267ms/step - loss: 0.4838 - accuracy: 0.7757 - val_loss: 0.3747 - val_accuracy: 0.8398
Epoch 2/30
381/381 [==============================] - ETA: 0s - loss: 0.3619 - accuracy: 0.8553
Epoch 00002: val_accuracy did not improve from 0.83979
381/381 [==============================] - 96s 253ms/step - loss: 0.3619 - accuracy: 0.8553 - val_loss: 0.4158 - val_accuracy: 0.8260
Epoch 3/30
381/381 [==============================] - ETA: 0s - loss: 0.3091 - accuracy: 0.8849
Epoch 00003: val_accuracy improved from 0.83979 to 0.84045, saving model to model.h5
381/381 [==============================] - 101s 266ms/step - loss: 0.3091 - accuracy: 0.8849 - val_loss: 0.3725 - val_accuracy: 0.8404
Epoch 4/30
381/381 [==============================] - ETA: 0s - loss: 0.2414 - accurac

Make predictions and submit results.

## Observations
1. model overfits quickly, `train_accuracy` reaches 0.9204 in 4 epochs, while `val_accuracy` is dropping 

In [32]:
%%time 
model.load_weights('model.h5')
test_pred = model.predict(test_input)
sub['target'] = test_pred.round().astype(int)
sub.to_csv('tweets.csv', index=False)

CPU times: user 2.54 s, sys: 944 ms, total: 3.49 s
Wall time: 16.1 s


In [0]:
! curl -X PUT --upload-file tweets.csv ali.140714.xyz:8000/


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 22746    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

## Received: "tweets.csv"

100 22775    0    29  100 22746     18  14646  0:00:01  0:00:01 --:--:-- 14665
